In [ ]:
import h5py
import numpy as np

ens='cD96'
ensemble='cD211.054.96'

path='/p/project/pines/li47/code/projectData/NST_b-discNJN/{}/data_merge/NST_b.h5_{}'.format(ensemble,ens)
path2='/p/project/pines/li47/code/projectData/NST_b-discNJN-old/data_merge/NST_b.h5_main'
outfile='dat/NST_b_{}.h5'.format(ens)
with h5py.File(path) as f, h5py.File(outfile,'w') as fw, h5py.File(path2) as f2:
    cfgs1=[cfg.decode() for cfg in f['cfgs']]; 
    cfgs2=[cfg.decode() for cfg in f2[ensemble+'/cfgs']]; 
    cfgs=list(set(cfgs1).intersection(set(cfgs2)))
    cfgs.sort()
    print(len(cfgs),len(cfgs1),len(cfgs2))
    
    dic={}
    for i,cfg in enumerate(cfgs1):
        dic[cfg]=i
    cfgs1_map=[dic[cfg] for cfg in cfgs]    
    dic={}
    for i,cfg in enumerate(cfgs2):
        dic[cfg]=i
    cfgs2_map=[dic[cfg] for cfg in cfgs]   

    key='VEV/j/data/j+'; fw.create_dataset(key,data=f[key][cfgs1_map])
    key='VEV/j/inserts'; fw.create_dataset(key,data=f[key])
    key='cfgs'; fw.create_dataset(key,data=cfgs)
    key='diags/N/data/N2_N2'; fw.create_dataset(key,data=f[key][cfgs1_map])
    key='diags/N/opabs'; fw.create_dataset(key,data=f[key])

    opabs=[opab.decode() for opab in f['diags/N-j/opabs']]; dic={}
    for i,opab in enumerate(opabs):
        dic[opab]=i
    opabs_new=['g;0,0,0;G1g;a;l1;N_g;0,0,0;G1g;a;l1;N','g;0,0,0;G1g;a;l2;N_g;0,0,0;G1g;a;l2;N']
    opabs_map=[dic[opab] for opab in opabs_new]
    
    inserts=[insert.decode() for insert in f['diags/N-j/inserts']]; dic={}
    for i,insert in enumerate(inserts):
        dic[insert]=i
    inserts_new=['id']
    inserts_map=[dic[insert] for insert in inserts_new]
    
    tfs=[int(key.split('_')[-1]) for key in f['diags/N-j/data'].keys()]
    tfs=list(set(tfs))
    tfs.sort()
    key='diags/N-j/opabs'; fw.create_dataset(key,data=opabs_new)
    key='diags/N-j/inserts'; fw.create_dataset(key,data=inserts_new)
    for tf in tfs:
        key='diags/N-j/data/N2_j+_N2_deltat_{}'.format(tf)
        t=f[key][:]
        t=t[cfgs1_map]
        t=t[:,:,opabs_map]
        t=t[:,:,:,inserts_map]
        fw.create_dataset(key,data=t)

    tfs=[int(key.split('_')[-1]) for key in f2[ensemble+'/diags/NJN/data/'].keys()]
    tfs=list(set(tfs))
    tfs.sort()
    for tf in tfs:
        key='diags/NJN/data/gS_j+_deltat_{}'.format(tf)
        t=f2[ensemble+'/'+key][:]
        t=t[cfgs2_map]
        fw.create_dataset(key,data=t)
